<a href="https://colab.research.google.com/github/dquerales/DataScience/blob/main/DataEngineering/ETL/notebooks/Extract_Load_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL

## Load libraries

In [1]:
import glob
import pandas as pd
from datetime import datetime

## Connect to S3

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2024-02-18 19:14:21--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.3’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2024-02-18 19:14:22 (1.53 GB/s) - ‘bank_market_cap_1.json.3’ saved [2815/2815]

--2024-02-18 19:14:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

## Functions

In [3]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [4]:
def extract():
    json_files = glob.glob("*.json") #returns list of files with .json extension
    extracted_data = extract_from_json(json_files[0])
    return extracted_data

In [5]:
def transform(data, rate):
    data["Market Cap (US$ Billion)"] = data["Market Cap (US$ Billion)"].apply(lambda x: x*rate) #converts currency from USD to GBP (GBP=USD*exchange_rate)
    data["Market Cap (US$ Billion)"] = round(data.loc[:,"Market Cap (US$ Billion)"] , 3)
    data.rename(columns = {'Market Cap (US$ Billion)':'Market Cap (GBP$ Billion)'}, inplace = True)
    return data

In [6]:
def load(target_file, data_to_load):
    data_to_load.to_csv(target_file)

In [7]:
log_file = "log_file.txt"

def log(message):
    # Write your code here
    format_ = "%H:%M:%S:%h:%d:%y"
    current_time = datetime.now()
    timestamp = current_time.strftime(format_)
    with open(log_file, "a") as f:
        f.write(timestamp + ", " + message + "\n\n")

## Extract

In [8]:
log("ETL Job Started")

In [9]:
log("Extract phase started")
extracted_data = extract()

In [10]:
extracted_data

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
65,Ping An Bank,37.993
66,Standard Chartered,37.319
67,United Overseas Bank,35.128
68,QNB Group,33.560


## Transform

In [11]:
rates = pd.read_csv('/content/exchange_rates.csv', index_col = 0)
exchange_rate = rates.loc["GBP"]

In [12]:
log("Transform phase Started")
transformed_data = transform(extracted_data, exchange_rate)

In [13]:
transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


## Load

In [14]:
target_file = "bank_market_cap_gbp.csv"

In [15]:
log("Load phase Started")
load(target_file, transformed_data)

## Log

In [16]:
pd.read_csv("log_file.txt")

,19:09:54:Feb:18:24,ETL Job Started
0,19:09:54:Feb:18:24,Extract phase started
1,19:09:55:Feb:18:24,Transform phase Started
2,19:09:55:Feb:18:24,Load phase Started
3,19:10:39:Feb:18:24,ETL Job Started
4,19:10:39:Feb:18:24,Extract phase started
5,19:10:51:Feb:18:24,ETL Job Started
6,19:10:51:Feb:18:24,Extract phase started
7,19:11:47:Feb:18:24,Transform phase Started
8,19:11:53:Feb:18:24,Load phase Started
9,19:12:57:Feb:18:24,ETL Job Started
